# Expert System (i.e. Knowledge-Based System) Shell/Engine - Educational DEMO
Author: Ryan J Urbanowicz, PhD

Institution: Cedars Sinai Health Sciences University

Date: 1/16/2026

Details: V4 - Extends backward chaining as a 'hybrid' option, also running forward chaining

This notebook includes code assembling a **basic but flexible expert system shell, built from the ground up** (i.e. without using existing expert system shells like CLIPS, clipspy, or PyKE), but certainly directly inspired by them. 

This expert system shell is primarily intended for educational purposes. While it is a realtively simple implementation it has been designed with a good deal of flexibility - not found in other Python or Java expert system implementations which are often confusing, not well documented, and or no longer supported/used often. This shell can be used to create a fairly wide varity of expert and knowledge-based system decision making/reasoning tools. Here is a summary of the key features of this shell impelemntation. 
1. Handles both **deductive** (certain) and **inductive** (uncertain/probabilistic) **reasoning**
2. Can perform **forward**, **backward**, and **hybrid chaining** (i.e. reasoning)
3. Separates the knowledge base (saved as a json file) from the inference engine and other components (a simple knowlege base editor and a simple explanation system)
4. Uses an easy-to-understand and simple syntax for facts, rules, and questions in the knoweldge-base (however potentially limiting for some applications)
5. Flexibly handles truth comparisons including **(==, >, <, <=, >=,!=)** and fact-states such as True/False, yes/no, etc. 
6. For inductive reasoning - employs certainty factores (0-1 values) - rule firing propagates uncertainty with a product of cfs, while 'and's' of conjuntive rules take the minimum cfs across rule conditions. When multiple rules fire, certainty factors are combined with the Mycin cf update (i.e. new_cf = new_cf + old_cf * (1 - new_cf))
7. The code in the shell below has also been adapted into a 'Streamlit' web dashboard/GUI that can be easily shared and played with. (i.e. **'expert_system_app.py'**)
    * This dashboard is:
        * Run with the command: **'streamlit run expert_system_app.py'**
        * Limited to **backward chaining**.
        * Allows loading of different knowlege bases as .json files.
        * Requires selection of **deductive** vs. **probailistic** (i.e. inductive) reasoning before loading a knowelege base.
        * Supports selection of certainty factors for inputs using a slider.
        * Allows selection from available goals in knowledge base that can be proved.
        * Provides a breakdown of reasoning, and an explanation of decisions.

---
## Expert System Shell
This shell does not be edited in order to build an expert system, however since this shell is simply and concisely built from the ground up, there are many opportunities to improve and extend this implementation to the needs of the user. 

In [24]:
import operator
import json

# Custom exception to break out of deep recursion/loops instantly
class GoalReachedException(Exception):
    def __init__(self, cf):
        self.cf = cf

class KnowledgeManager:
    @staticmethod
    def load_from_json(engine, file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)

        for fact_name, text in data.get("questions", {}).items():
            engine.add_question(fact_name, text)

        for f_data in data.get("facts", []):
            try:
                engine.initialize_fact(f_data["name"], Fact(
                    f_data["name"], f_data["value"], f_data["cf"], f_data["explanation"]
                ))
            except:
                engine.initialize_fact(f_data["name"], Fact(
                    f_data["name"], f_data["value"], 1.0, f_data["explanation"]
                ))

        for r_data in data.get("rules", []):
            conditions = [Condition(c["fact1"], c["op"], c["fact2"]) for c in r_data["conditions"]]
            try:
                engine.add_rule(Rule(
                    r_data["id"], conditions, tuple(r_data["conclusion"]), 
                    r_data["cf"], r_data["explanation"]
                ))
            except:
                engine.add_rule(Rule(
                    r_data["id"], conditions, tuple(r_data["conclusion"]), 
                    1.0, r_data["explanation"]
                ))
        print(f"Successfully loaded {len(data['facts'])} facts, {len(data['rules'])} rules.")

# ... [Condition, Fact, Rule classes remain the same] ...
class Condition:
    def __init__(self, fact1, op, fact2):
        self.fact1, self.op, self.fact2 = fact1, op, fact2 
    def evaluate(self, f1_v, f2_v):
        return OPERATORS[self.op](f1_v, f2_v)

class Fact:
    def __init__(self, name, value, cf=1.0, explanation="Given", dependencies=None):
        self.name, self.value, self.cf, self.explanation = name, value, cf, explanation
        self.dependencies = dependencies if dependencies else []

class Rule:
    def __init__(self, id, conditions, conclusion, rule_cf, explanation):
        self.id, self.conditions, self.conclusion, self.rule_cf, self.explanation = id, conditions, conclusion, rule_cf, explanation

OPERATORS = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le, "==": operator.eq, "!=": operator.ne}

class ExpertSystem:
    def __init__(self, reasoning='deductive'):
        self.rules, self.facts, self.questions = [], {}, {}
        self.reasoning = reasoning
        self.target_goal = None  # Track the current top-level goal

    def add_fact(self, name, value, cf=1.0, explanation="Initial", dependencies=None):
        if name in self.facts: 
            old_cf = self.facts[name].cf
            self.facts[name].cf = old_cf + cf * (1 - old_cf)
            if dependencies: self.facts[name].dependencies.extend([d for d in dependencies if d not in self.facts[name].dependencies])
        else:
            self.facts[name] = Fact(name, value, cf, explanation, dependencies)
        
        # IMMEDIATELY STOP if the fact being added is our target goal
        if name == self.target_goal and self.facts[name].cf > 0:
            raise GoalReachedException(self.facts[name].cf)

    def clear_facts(self):self.facts = {}
    def add_rule(self, r): self.rules.append(r)
    def add_question(self, k, v): self.questions[k] = v
    def initialize_fact(self, n, f): self.facts[n] = f

    def get_explanation(self, fact_name):
        if fact_name not in self.facts:
            return "Fact ("+str(fact_name)+") not found."
        
        target_fact = self.facts[fact_name]
        lines = [f"Conclusion: {target_fact.name} is {target_fact.value} (Confidence: {target_fact.cf:.2%})"]
        lines.append("Reasoning Chain:")
        
        seen = set()
        
        def walk(name, level):
            if name in seen: return
            seen.add(name)
            f = self.facts.get(name)
            if not f: return
            indent = "  " * level
            marker = "└─" if level > 0 else "•"
            lines.append(f"{indent}{marker} {f.name}: {f.value} (CF={f.cf:.2f}) -- [{f.explanation}]")
            for dep in f.dependencies:
                walk(dep, level + 1)
        
        walk(fact_name, 0)
        return "\n".join(lines)
    
    # --- FORWARD CHAINING ---
    def forward_chain(self, verbose=True, hybrid=False):
        if verbose and hybrid==False: print("\n--- Forward Chaining Inference Cycle ---")
        fired_rules = set()
        changed = True
        while changed:
            changed = False
            for rule in self.rules:
                if rule.id in fired_rules: continue
                satisfied, min_cf, deps = True, 1.0, []
                for cond in rule.conditions:
                    f1 = self.facts.get(cond.fact1.removeprefix('$')) if cond.fact1.startswith('$') else Fact(cond.fact1, cond.fact1)
                    f2 = self.facts.get(cond.fact2.removeprefix('$')) if cond.fact2.startswith('$') else Fact(cond.fact2, cond.fact2)
                    if f1 and f2 and cond.evaluate(f1.value, f2.value):
                        min_cf = min(min_cf, f1.cf, f2.cf)
                        if cond.fact1.startswith('$'): deps.append(f1.name)
                        if cond.fact2.startswith('$'): deps.append(f2.name)
                    else: satisfied = False; break
                if satisfied:
                    # add_fact will raise GoalReachedException if conclusion == target_goal
                    self.add_fact(rule.conclusion[0], rule.conclusion[1], rule.rule_cf * min_cf, rule.explanation, deps)
                    if verbose: print(f"  Rule Fired: {rule.id} -> {rule.conclusion[0]} is {rule.conclusion[1]} with cf {rule.rule_cf * min_cf:.2%}")
                    fired_rules.add(rule.id); changed = True


    # --- BACKWARD CHAINING ---
    def backward_chain(self, goal_name, hybrid=False):
        self.target_goal = goal_name
        print(f"\n--- Reasoning for: {goal_name} ---")
        if goal_name in self.facts: return self.facts[goal_name].cf
        try:
            return self._prove(goal_name,hybrid)
        except GoalReachedException as e:
            print(f"Goal '{goal_name}' proved early. Terminating search.")
            return e.cf

    def hybrid_chain(self, goal_name, hybrid=True):
        self.target_goal = goal_name
        print(f"\n--- Reasoning for: {goal_name} ---")
        if goal_name in self.facts: return self.facts[goal_name].cf
        try:
            return self._prove(goal_name,hybrid)
        except GoalReachedException as e:
            print(f"Goal '{goal_name}' proved early. Terminating search.")
            return e.cf

    def _prove(self, goal_name, hybrid):
        if goal_name in self.facts: return self.facts[goal_name].cf

        # 1. Attempt rule proof FIRST (Check without asking)
        rule_cfs = []
        for rule in self.rules:
            if rule.conclusion[0] == goal_name:
                satisfied, cond_cfs, deps = True, [], []
                for cond in rule.conditions:
                    c1_cf = self._prove(cond.fact1.removeprefix('$'),hybrid) if cond.fact1.startswith('$') else 1.0
                    c2_cf = self._prove(cond.fact2.removeprefix('$'),hybrid) if cond.fact2.startswith('$') else 1.0
                    f1_v = self.facts[cond.fact1.removeprefix('$')].value if cond.fact1.startswith('$') else cond.fact1
                    f2_v = self.facts[cond.fact2.removeprefix('$')].value if cond.fact2.startswith('$') else cond.fact2
                    
                    if min(c1_cf, c2_cf) > 0 and OPERATORS[cond.op](f1_v, f2_v):
                        cond_cfs.append(min(c1_cf, c2_cf))
                        if cond.fact1.startswith('$'): deps.append(cond.fact1.removeprefix('$'))
                        if cond.fact2.startswith('$'): deps.append(cond.fact2.removeprefix('$'))
                    else: satisfied = False; break
                
                if satisfied:
                    new_cf = rule.rule_cf * min(cond_cfs)
                    rule_cfs.append(new_cf)
                    # add_fact here might trigger GoalReachedException
                    self.add_fact(goal_name, rule.conclusion[1], new_cf, rule.explanation, deps)

        if rule_cfs:
            res = rule_cfs[0]
            for c in rule_cfs[1:]: res = res + c * (1 - res)
            return res

        # 2. Only if rules failed, ask User
        if goal_name in self.questions:
            print(f"QUERY: {self.questions[goal_name]}")
            val_in = input("Value: ")
            try: val = float(val_in)
            except: val = val_in.lower().strip()
            conf = 1.0 if self.reasoning == 'deductive' else float(input("Confidence (0-1): "))
            
            self.add_fact(goal_name, val, conf, "User provided")
            if hybrid:
                # 3. After answer, run recursive Forward Chain (Inference Cycle)
                self.forward_chain(verbose=True, hybrid=hybrid)
            else:
                pass
            return self.facts[goal_name].cf
            
        return 0.0


---
## Introduction to Knowledge Base Syntax
Below we explore simple expert systems designed as accessible examples.
* Each knowlege base is stored within a .json file. 
* The .json knowlege base syntax is that of a dictionary organized by 'questions', 'facts' and 'rules'. Each type of entry has its own required syntax.
    * 'questions' - a dictionary of key:value pairs where the key is the name of a fact needed by the system and the value is the text description of the input needed seen by the user.
    * 'facts' - an array of objects (i.e. a list of dictionaries) where each fact has the keys (name, value, cf, explanation) identifying the fact name, it's assigned value, it's certainty factor, and a description of the fact (used by the explanation system)
    * 'rules' - an array of objects (i.e. a list of dictionaries) where each rule has the keys (id, conditions, conclusion, cf, explanation) 
        * 'Conditions' are list objects that can include one or more conditions (to be satisfied) in order for the rule to 'fire' (i.e. add the conclusion as a new fact in the knowledge base)
        * Each condition has two 'facts' that are compared using any of the following (==, >, <, <=, >=,!=). 
        * These 'facts' can either be variables (starting with '$') (which can also be defined as facts in the KB directly) or static (hard-coded) values within the condition itself, that the system will automatically turn into 'implied facts'. 
        * 'Conclusions' are given as a tuple that first defines the [fact name, fact value] when the rule fires.
        * 'cf' and 'explanation' mean the same as for 'facts'

---
### Weather Knowlege Base (Example) - Deductive Reasoning - Forward Chaining
This is a simple expert system designed to provide users guidance on 'what to bring' when leaving the house based on current weather conditions.
* This knowledge base can be found in 'kb_weather.json'.
* In this forward chaining example, only facts and rules are used (not questions).
* Neither facts, rules, nor questions include certainty factors in this knowledge base, so the system will assume they all have a certainty factor of 1.0 by default.

The code below applies 'forward chaining' reasoning to this simple deductive reasoning example knowledge base.


In [12]:
# 1. Initialize the empty engine
engine = ExpertSystem('deductive') #deductive or probabilistic

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_weather.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Manually add some facts for testing forward chaining (i.e. part of the simple knowlege base editor)
engine.add_fact("precipitation", "yes")
engine.add_fact("windy", "yes") 
engine.add_fact("temperature", 40)

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "what to bring"

# 3. Use the system for a decision making
print("\n--- AUTOMATED DECISION MAKING ---")

# Forward chaining will now use the rules and facts loaded from JSON
engine.forward_chain()

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

#print(engine.facts) #optionally print all facts stored in the knowlege base at the end of forward chaining
engine.clear_facts() #cleanup expert system fact memory for another run

Successfully loaded 3 facts, 9 rules.

--- AUTOMATED DECISION MAKING ---

--- Forward Chaining Inference Cycle ---
  Rule Fired: R_RAIN_PROTECT -> rain_protect is True with cf 100.00%
  Rule Fired: R_RAINCOAT -> what to bring is raincoat with cf 100.00%
  Rule Fired: R_WARMCOAT2 -> what to bring is warm coat with cf 100.00%
  Rule Fired: R_WARMCOAT3 -> what to bring is warm coat with cf 100.00%

Conclusion: what to bring is raincoat (Confidence: 100.00%)
Reasoning Chain:
• what to bring: raincoat (CF=1.00) -- [Bring raincoat when it's raining and windy.]
  └─ rain_protect: True (CF=1.00) -- [Rain protection needed when it's raining.]
    └─ precipitation: yes (CF=1.00) -- [Initial]
    └─ temperature: 40 (CF=1.00) -- [Initial]
    └─ freeze_threshold: 0.0 (CF=1.00) -- [Threshold for water freezing (in Fahrenheit).]
  └─ windy: yes (CF=1.00) -- [Initial]
  └─ windchill_threshold: 77.0 (CF=1.00) -- [Threshold for 'chilly' temperature with wind (in Fahrenheit).]
  └─ chilly_threshold: 65.

---
### Weather Knowlege Base (Example) - Deductive Reasoning - Backward Chaining
The code below applies 'backward chaining' reasoning to this simple deductive reasoning example knowledge base. When run, a prompt will appear asking for user inputs generated by the questions built into the knowlege base. This example will use everything in the knowledge base, i.e. facts, rules, and questions.

* Upon request by the system enter in the following facts...
    * Is it currently precipitating? (yes/no) - <font color="red">yes</font>
    * What is the current temperature (in Fahrenheit)? - <font color="red">80</font>
    * Is it currently windy? (yes/no) - <font color="red">no</font>

In [13]:
# 1. Initialize the empty engine
engine = ExpertSystem('deductive')

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_weather.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "what to bring"

# 3. Use the system for a diagnosis
print("\n--- MODULAR DIAGNOSTIC SESSION ---")
# Backward chaining will now use the rules and questions loaded from JSON
engine.backward_chain(goal_name)

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

Successfully loaded 3 facts, 9 rules.

--- MODULAR DIAGNOSTIC SESSION ---

--- Reasoning for: what to bring ---
QUERY: Is it currently precipitating? (yes/no)
QUERY: What is the current temperature (in Fahrenheit)
QUERY: Is it currently windy? (yes/no)
Goal 'what to bring' proved early. Terminating search.

Conclusion: what to bring is umbrella (Confidence: 100.00%)
Reasoning Chain:
• what to bring: umbrella (CF=1.00) -- [Bring an umbrella when it's raining but not windy.]
  └─ rain_protect: True (CF=1.00) -- [Rain protection needed when it's raining.]
    └─ precipitation: yes (CF=1.00) -- [User provided]
    └─ temperature: 80.0 (CF=1.00) -- [User provided]
    └─ freeze_threshold: 0.0 (CF=1.00) -- [Threshold for water freezing (in Fahrenheit).]
  └─ windy: no (CF=1.00) -- [User provided]


---
### Weather Knowlege Base (Example) - Deductive Reasoning - Hybrid Chaining
The code below applies 'hybrid chaining' reasoning to this simple deductive reasoning example knowledge base. When run, a prompt will appear asking for user inputs generated by the questions built into the knowlege base. This example will use everything in the knowledge base, i.e. facts, rules, and questions.

* Upon request by the system enter in the following facts...
    * Is it currently precipitating? (yes/no) - <font color="red">yes</font>
    * What is the current temperature (in Fahrenheit)? - <font color="red">80</font>
    * Is it currently windy? (yes/no) - <font color="red">no</font>

In [29]:
# 1. Initialize the empty engine
engine = ExpertSystem('deductive')

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_weather.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "what to bring"

# 3. Use the system for a diagnosis
print("\n--- MODULAR DIAGNOSTIC SESSION ---")
# Backward chaining will now use the rules and questions loaded from JSON
engine.hybrid_chain(goal_name)

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

Successfully loaded 3 facts, 13 rules.

--- MODULAR DIAGNOSTIC SESSION ---

--- Reasoning for: what to bring ---
QUERY: Is it currently precipitating? (yes/no)
  Rule Fired: R_RAIN_PROTECT2 -> rain_protect is False with cf 100.00%
  Rule Fired: R_SNOW_PROTECT2 -> snow_protect is False with cf 100.00%
QUERY: Is it currently windy? (yes/no)
  Rule Fired: R_RAIN_PROTECT2 -> rain_protect is False with cf 100.00%
  Rule Fired: R_SNOW_PROTECT2 -> snow_protect is False with cf 100.00%
QUERY: What is the current temperature (in Fahrenheit)
  Rule Fired: R_RAIN_PROTECT2 -> rain_protect is False with cf 100.00%
  Rule Fired: R_SNOW_PROTECT2 -> snow_protect is False with cf 100.00%
Goal 'what to bring' proved early. Terminating search.

Conclusion: what to bring is warm coat (Confidence: 100.00%)
Reasoning Chain:
• what to bring: warm coat (CF=1.00) -- [Bring warm coat when it's chilly.]
  └─ temperature: -10.0 (CF=1.00) -- [User provided]
  └─ chilly_threshold: 65.0 (CF=1.00) -- [Threshold for '

---
### Simple Diagnosis Knowlege Base (Example) - Inductive Reasoning - Forward Chaining
This is a simple expert system designed to provide users guidance on 'diagnosis' of flu.
* This knowledge base can be found in 'kb_diagnose_flu.json'.
* In this forward chaining example, only facts and rules are used (not questions).
* Facts, and rules include certainty factors in this knowledge base denoted by 'cf'.
* Users will input certainty factors along with any input facts added at the start of forward chaining

The code below applies 'forward chaining' reasoning to this simple inductive reasoning example knowledge base.

In [14]:
# 1. Initialize the empty engine
engine = ExpertSystem('probabilistic') #deductive or probabilistic

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_diagnose_flu.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Manually add some facts for testing forward chaining (i.e. part of the simple knowlege base editor)
engine.add_fact("temperature", 101, 1.0)
engine.add_fact("cough", "yes", 1.0) 

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "diagnosis"

# 3. Use the system for a decision making
print("\n--- AUTOMATED DECISION MAKING ---")

# Forward chaining will now use the rules and facts loaded from JSON
engine.forward_chain()

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

#print(engine.facts) #optionally print all facts stored in the knowlege base at the end of forward chaining
engine.clear_facts() #cleanup expert system fact memory for another run

Successfully loaded 1 facts, 2 rules.

--- AUTOMATED DECISION MAKING ---

--- Forward Chaining Inference Cycle ---
  Rule Fired: R_FEVER -> status is feverish with cf 90.00%
  Rule Fired: R_FLU -> diagnosis is influenza with cf 72.00%

Conclusion: diagnosis is influenza (Confidence: 72.00%)
Reasoning Chain:
• diagnosis: influenza (CF=0.72) -- [Fever combined with a cough suggests Influenza.]
  └─ status: feverish (CF=0.90) -- [Temperature above clinical threshold is a fever.]
    └─ temperature: 101 (CF=1.00) -- [Initial]
    └─ threshold: 100.0 (CF=1.00) -- [Clinical threshold defining fever is 100 F.]
  └─ cough: yes (CF=1.00) -- [Initial]


---
### Simple Diagnosis Knowlege Base (Example) - Inductive Reasoning - Backward Chaining
The code below applies 'backward chaining' reasoning to this simple deductive reasoning example knowledge base. When run, a prompt will appear asking for user inputs generated by the questions built into the knowlege base. This example will use everything in the knowledge base, i.e. facts, rules, and questions.
* Users will input certainty factors (cf) along with any input questions asked at the start of backward chaining

The code below applies 'backward chaining' reasoning to this simple inductive reasoning example knowledge base.

* Upon request by the system enter in the following facts...
    * What is the patient's temperature in Farenheit? - <font color="red">102</font>
        * Cofidence (0-1) - <font color="red">0.8</font>
    * Does the patient have a persistent cough? - <font color="red">yes</font>
        * Cofidence (0-1) - <font color="red">1.0</font>

In [18]:
# 1. Initialize the empty engine
engine = ExpertSystem('probabilistic')

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_diagnose_flu.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "diagnosis"

# 3. Use the system for a diagnosis
print("\n--- MODULAR DIAGNOSTIC SESSION ---")
# Backward chaining will now use the rules and questions loaded from JSON
engine.backward_chain(goal_name)

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

Successfully loaded 1 facts, 2 rules.

--- MODULAR DIAGNOSTIC SESSION ---

--- Reasoning for: diagnosis ---
QUERY: What is the patient's temperature in Farenheit?
QUERY: Does the patient have a persistent cough? (yes/no)
Goal 'diagnosis' proved early. Terminating search.

Conclusion: diagnosis is influenza (Confidence: 57.60%)
Reasoning Chain:
• diagnosis: influenza (CF=0.58) -- [Fever combined with a cough suggests Influenza.]
  └─ status: feverish (CF=0.72) -- [Temperature above clinical threshold is a fever.]
    └─ temperature: 102.0 (CF=0.80) -- [User provided]
    └─ threshold: 100.0 (CF=1.00) -- [Clinical threshold defining fever is 100 F.]
  └─ cough: yes (CF=1.00) -- [User provided]
